# Lasso and Ridge Regression Implementation
In this Notebook, we implement Lasso (L1 Regularization) and Ridge regression (L2 regularization) from scratch, and compare it with the pre-defined models from the Scikit-learn library.

### Initial functions

In [295]:
# Importing libraries 
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Most formulations are same in both Lasso and Ridge regression. From an implementation perspective, they differ solely in the regularization term. Hence, we implement a common regression class, in which L1 or L2 regularization can be chosen as a parameter.

In [296]:
class Regression:
    
    def __init__(self, regularization, lr, epoch):
        self.m = None #samples
        self.n = None #features
        self.w = None #weight
        self.b = None #bias
        self.regularization = regularization #penalty object
        self.lr = lr #learning rate
        self.epoch = epoch #iteration
        
    def __calculate_cost(self, y, y_pred):
        return (1 / (2*self.m)) * np.sum(np.square(y_pred-y)) + self.regularization(self.w)
    
    def __hypothesis(self, w, X):
        return np.dot(X, w) 
    
    def __initialization(self, X):
        X = np.insert(X, 0, 1, axis=1)
        self.m, self.n = X.shape
        self.w = np.zeros((self.n,1))
        return X
    
    def __update_parameters(self, X, y, y_pred):
        dw = (1/self.m) * np.dot(X.T, (y_pred - y)) + self.regularization.derivation(self.w)
        self.w = self.w - self.lr * dw
        return True
        
    def fit(self, X, y):
        X = self.__initialization(X)
        for e in range(1, self.epoch+1):
            y_pred = self.__hypothesis(self.w, X)
            cost = self.__calculate_cost(y, y_pred)
            self.__update_parameters(X, y, y_pred)
            if e % 100 == 0:
                print(f"The Cost in iteration {e}----->{cost} :)")
        return True

    def predict(self, X_test):
        X_test = np.insert(X_test, 0 , 1, axis= 1)
        y_pred = self.__hypothesis(self.w, X_test)
        return y_pred

Define the regularization terms for Lasso and Ridge respectively:

In [297]:
class LassoPenalty:
    
    def __init__(self, l):
        self.l = l # lambda value
        
    def __call__(self,w):
        return self.l * np.sum(np.abs(w))
        
    def derivation(self, w):
        return self.l * np.sign(w)
    
class RidgePenalty:
    
    def __init__(self, l):
        self.l = l
        
    def __call__(self, w):
        return self.l * np.sum(np.square(w))
        
    def derivation(self, w):
        return self.l * 2 * w

In [298]:
class Lasso(Regression):
    
    def __init__(self, l, lr, epoch):
        self.regularization = LassoPenalty(l)
        super().__init__(self.regularization, lr, epoch )
        
class Ridge(Regression):
    
    def __init__(self, l, lr, epoch):
        self.regularization = RidgePenalty(l)
        super().__init__(self.regularization, lr, epoch )

### Dataset Operations 1

In [299]:
# Importing dataset     
df = pd.read_csv( "../ML-Project-CS361/cleaned_shifted_data.csv" ) 
drop_cols = [0,1,2,12,14,16]
drop_cols = df.columns[drop_cols]
drop_cols # Dropping unecessary columns

Index(['Timestamp', 'Unnamed: 0', 'Station', 'Checks', 'AQI_bucket_calculated',
       'AQI_bucket_calculated_shifted'],
      dtype='object')

In [300]:
# Drop the columns and make the changes in-place
df.drop(columns=drop_cols, inplace=True)

In [301]:
print(df.shape)  # The dataset has a size of 1,74,762 records, 10 features, 1 target variable

(174762, 11)


In [302]:
df.head() 

,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),AQI_calculated,AQI_calculated_shifted
0,46.00,80.00,1.29,9.16,12.02,27.19,13.56,0.40,15.80,67.0,296.0
1,46.00,80.00,1.74,8.93,12.48,30.29,13.71,0.41,15.52,68.0,297.0
2,45.62,79.92,1.87,8.56,12.17,28.20,13.88,0.41,15.33,68.0,298.0
3,41.00,72.92,1.83,8.72,12.37,26.69,13.77,0.40,15.30,68.0,298.0
4,41.00,79.00,1.69,7.91,11.30,26.83,13.87,0.41,15.49,68.0,299.0


Since the original dataset is too large to fit into a numpy array, take a random subset of this data. 

In [303]:
# Randomly sample 32000 rows
df_subset = df
print(df_subset.shape)

(174762, 11)


In [304]:
df_subset.describe()

,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),AQI_calculated,AQI_calculated_shifted
count,174762.000000,174762.000000,174762.000000,174762.000000,174762.000000,174762.000000,174762.000000,174762.000000,174762.000000,174762.000000,174762.000000
mean,59.679503,111.238568,9.660644,8.324557,17.762988,9.984327,18.461032,0.689923,24.271172,140.573117,139.053112
std,59.876848,111.774626,20.843351,10.595687,33.777433,7.575767,13.741348,0.625532,22.874962,104.720841,105.081092
min,0.100000,1.590000,0.010000,0.020000,0.000000,0.010000,0.100000,0.000000,0.010000,14.000000,11.000000
25%,20.000000,37.670000,1.210000,2.520000,4.590000,4.000000,10.080000,0.310000,12.670000,56.000000,56.000000
50%,39.000000,73.000000,3.360000,4.170000,5.450000,7.200000,14.460000,0.490000,18.080000,101.000000,100.000000
75%,81.000000,148.000000,5.597500,10.080000,13.280000,15.420000,22.770000,0.840000,26.710000,214.000000,204.000000
max,923.080000,1000.000000,472.550000,122.000000,488.620000,113.300000,195.010000,9.710000,193.570000,1109.000000,1109.000000


In [305]:
# Separating the features and labels/target variables
X = df_subset.drop('AQI_calculated_shifted',axis = 1)  # feature set
Y = df_subset['AQI_calculated_shifted'] # target variable
print(X.shape)
print(Y.shape)

(174762, 10)
(174762,)


In [306]:
X # it is a pandas dataframe

,PM2.5 (µg/m³),PM10 (µg/m³),NO (µg/m³),NO2 (µg/m³),NOx (ppb),NH3 (µg/m³),SO2 (µg/m³),CO (mg/m³),Ozone (µg/m³),AQI_calculated
0,46.00,80.00,1.29,9.16,12.02,27.19,13.56,0.40,15.80,67.0
1,46.00,80.00,1.74,8.93,12.48,30.29,13.71,0.41,15.52,68.0
2,45.62,79.92,1.87,8.56,12.17,28.20,13.88,0.41,15.33,68.0
3,41.00,72.92,1.83,8.72,12.37,26.69,13.77,0.40,15.30,68.0
4,41.00,79.00,1.69,7.91,11.30,26.83,13.87,0.41,15.49,68.0
...,...,...,...,...,...,...,...,...,...,...
174757,72.00,116.00,6.40,3.30,6.90,4.90,44.20,0.63,68.00,252.0
174758,71.00,114.00,6.40,3.40,7.00,4.90,41.30,0.68,69.10,249.0
174759,71.00,114.00,6.30,3.50,7.00,4.90,42.20,0.73,66.90,247.0
174760,73.00,114.00,6.30,5.00,7.80,5.50,0.40,0.76,44.60,238.0


In [307]:
Y # it is a pandas series

0         296.0
1         297.0
2         298.0
3         298.0
4         299.0
          ...  
174757    219.0
174758    219.0
174759    219.0
174760    220.0
174761    220.0
Name: AQI_calculated_shifted, Length: 174762, dtype: float64

In [308]:
Y = Y.values.reshape(-1, 1) # convert to a numpy array
Y 

array([[296.],
       [297.],
       [298.],
       ...,
       [219.],
       [220.],
       [220.]])

In [309]:
# Splitting dataset into train and test set 
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 1 / 3, random_state = 0 )
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(116508, 10)
(58254, 10)
(116508, 1)
(58254, 1)


In [310]:
Y_train

array([[356.],
       [260.],
       [120.],
       ...,
       [ 73.],
       [116.],
       [ 35.]])

In [311]:
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 3.67831173,  2.2237575 , -0.20599341, ...,  3.40249554,
         0.08160396,  1.92882892],
       [-0.29548835, -0.3588719 , -0.26667847, ..., -0.25689331,
        -0.23740736, -0.68081168],
       [ 1.56565852,  0.86541954, -0.35201684, ...,  0.5616542 ,
        -0.98845864,  1.75739267],
       ...,
       [-0.79850102, -0.71758036, -0.40464216, ..., -0.51369252,
        -0.34563555, -0.89034486],
       [-0.84880229, -0.76101143, -0.15526449, ...,  2.64814784,
        -0.94438184, -0.24269683],
       [-0.69370672, -0.52419593, -0.20220059, ..., -0.2247934 ,
        -0.31901491, -0.97606298]])

### L1 Regularization (Lasso)

Here we can observe much lower values of the cost function, since the regularization term depends only on the absolute values of weights.

In [312]:
parameters = {
    "l" : 0.5,
    "lr" : 0.1,
    "epoch" : 2000
}
model = Lasso(**parameters)
model.fit(X_train, Y_train) 

Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)  # calculate the R-squared error
print("r2 score", score)  
mse = mean_squared_error(Y_test, Y_pred)  # calculate the mean-squared error
print("MSE", mse)

The Cost in iteration 100----->3433.2781034400123 :)
The Cost in iteration 200----->3430.533352464965 :)
The Cost in iteration 300----->3430.1665350097896 :)
The Cost in iteration 400----->3430.099860545677 :)
The Cost in iteration 500----->3430.082388297635 :)
The Cost in iteration 600----->3430.084002260344 :)
The Cost in iteration 700----->3430.090547304069 :)
The Cost in iteration 800----->3430.0882620101647 :)
The Cost in iteration 900----->3430.0870978592006 :)
The Cost in iteration 1000----->3430.086501801884 :)
The Cost in iteration 1100----->3430.086194262193 :)
The Cost in iteration 1200----->3430.086034912643 :)
The Cost in iteration 1300----->3430.0859521606776 :)
The Cost in iteration 1400----->3430.0859091356765 :)
The Cost in iteration 1500----->3430.0858867518937 :)
The Cost in iteration 1600----->3430.0858751029305 :)
The Cost in iteration 1700----->3430.085869039553 :)
The Cost in iteration 1800----->3430.085865883236 :)
The Cost in iteration 1900----->3430.0858642401

### L2 Regularization (Ridge)

Here, we can observe much higher values of the cost function, as the regularization term depends on squares of weights.

In [313]:
parameters = {
    "l" : 0.001,
    "lr" : 0.1,
    "epoch" : 2000
}
model = Ridge(**parameters)
model.fit(X_train, Y_train)
 
Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)
print("r2 score", score)
mse = mean_squared_error(Y_test, Y_pred)
print("MSE", mse)

The Cost in iteration 100----->3323.094378975041 :)
The Cost in iteration 200----->3317.2272787105635 :)
The Cost in iteration 300----->3316.129089267472 :)
The Cost in iteration 400----->3315.852972390586 :)
The Cost in iteration 500----->3315.7813989338783 :)
The Cost in iteration 600----->3315.7627760481837 :)
The Cost in iteration 700----->3315.7579282347747 :)
The Cost in iteration 800----->3315.756666203188 :)
The Cost in iteration 900----->3315.7563376560256 :)
The Cost in iteration 1000----->3315.75625212462 :)
The Cost in iteration 1100----->3315.7562298580383 :)
The Cost in iteration 1200----->3315.7562240613292 :)
The Cost in iteration 1300----->3315.756222552259 :)
The Cost in iteration 1400----->3315.7562221593994 :)
The Cost in iteration 1500----->3315.7562220571253 :)
The Cost in iteration 1600----->3315.7562220305 :)
The Cost in iteration 1700----->3315.7562220235686 :)
The Cost in iteration 1800----->3315.756222021764 :)
The Cost in iteration 1900----->3315.75622202129

### Dataset Operations 2

In [314]:
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv("cleaned_shifted_data.csv")
    
oe = OneHotEncoder(sparse=False)
encoded = oe.fit_transform(pd.DataFrame(df['Station']))

In [315]:
oe.get_feature_names()

array(['x0_IITG ', 'x0_LGBI Airport ', 'x0_Pan Bazaar ',
       'x0_Railway Colony '], dtype=object)

In [316]:

one_hot_df = pd.DataFrame(encoded, columns=oe.get_feature_names())
df = pd.concat([df, one_hot_df], axis=1)

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['year'] = df['Timestamp'].dt.year
df['month'] = df['Timestamp'].dt.month
df['dayofweek'] = df['Timestamp'].dt.day_of_week

drop_cols = [0,1,2,12,14,16]
drop_cols = df.columns[drop_cols]
df.drop(drop_cols,axis=1,inplace=True)

In [317]:
df.shape

(174762, 18)

In [318]:
# # Take a subset
# df = df.iloc[:90000]
# df.shape

(174762, 18)

In [319]:
X = df.drop('AQI_calculated_shifted',axis = 1)
y = df['AQI_calculated_shifted'].values.reshape(-1, 1)
print(X.shape, y.shape)

(174762, 17) (174762, 1)


In [320]:
# Splitting dataset into train and test set 
X_train, X_test, Y_train, Y_test = train_test_split( X, y, test_size = 1 / 3, random_state = 0 )
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(116508, 17)
(58254, 17)
(116508, 1)
(58254, 1)


In [321]:
# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[ 3.67831173,  2.2237575 , -0.20599341, ...,  1.19263893,
        -1.54346737, -0.01566533],
       [-0.29548835, -0.3588719 , -0.26667847, ..., -0.95436545,
        -0.96118867, -0.01566533],
       [ 1.56565852,  0.86541954, -0.35201684, ..., -0.23869732,
        -1.25232802,  0.49471947],
       ...,
       [-0.79850102, -0.71758036, -0.40464216, ...,  0.47697081,
         0.49450809,  0.49471947],
       [-0.84880229, -0.76101143, -0.15526449, ...,  1.19263893,
         0.78564744,  1.00510426],
       [-0.69370672, -0.52419593, -0.20220059, ..., -0.95436545,
         0.49450809, -0.01566533]])

### Lasso 

In [322]:
parameters = {
    "l" : 0.5,
    "lr" : 0.1,
    "epoch" : 2000
}
model = Lasso(**parameters)
model.fit(X_train, Y_train) 

Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)  # calculate the R-squared error
print("r2 score", score)  
mse = mean_squared_error(Y_test, Y_pred)  # calculate the mean-squared error
print("MSE", mse)

The Cost in iteration 100----->3383.708721773161 :)
The Cost in iteration 200----->3380.691121394675 :)
The Cost in iteration 300----->3380.190894239744 :)
The Cost in iteration 400----->3380.0603291408447 :)
The Cost in iteration 500----->3380.012740963777 :)
The Cost in iteration 600----->3379.9885238702327 :)
The Cost in iteration 700----->3379.9830758064377 :)
The Cost in iteration 800----->3379.995480945607 :)
The Cost in iteration 900----->3379.988985296985 :)
The Cost in iteration 1000----->3379.9820166306195 :)
The Cost in iteration 1100----->3379.981877485505 :)
The Cost in iteration 1200----->3379.982803598071 :)
The Cost in iteration 1300----->3379.9890503636566 :)
The Cost in iteration 1400----->3379.9818922029904 :)
The Cost in iteration 1500----->3379.983397531838 :)
The Cost in iteration 1600----->3379.9880361885484 :)
The Cost in iteration 1700----->3379.983261964285 :)
The Cost in iteration 1800----->3379.986708093791 :)
The Cost in iteration 1900----->3379.99128242416

### Ridge

In [323]:
parameters = {
    "l" : 0.001,
    "lr" : 0.1,
    "epoch" : 2000
}
model = Ridge(**parameters)
model.fit(X_train, Y_train)
 
Y_pred = model.predict(X_test)
score = r2_score(Y_test, Y_pred)
print("r2 score", score)
mse = mean_squared_error(Y_test, Y_pred)
print("MSE", mse)

The Cost in iteration 100----->3255.3212191696716 :)
The Cost in iteration 200----->3247.776226824679 :)
The Cost in iteration 300----->3246.0273822807035 :)
The Cost in iteration 400----->3245.476114267615 :)
The Cost in iteration 500----->3245.296760879461 :)
The Cost in iteration 600----->3245.238210040228 :)
The Cost in iteration 700----->3245.2190887851884 :)
The Cost in iteration 800----->3245.2128440071497 :)
The Cost in iteration 900----->3245.210804526896 :)
The Cost in iteration 1000----->3245.2101384533166 :)
The Cost in iteration 1100----->3245.209920920425 :)
The Cost in iteration 1200----->3245.209849876383 :)
The Cost in iteration 1300----->3245.2098266741173 :)
The Cost in iteration 1400----->3245.209819096492 :)
The Cost in iteration 1500----->3245.2098166217165 :)
The Cost in iteration 1600----->3245.2098158134804 :)
The Cost in iteration 1700----->3245.209815549518 :)
The Cost in iteration 1800----->3245.209815463311 :)
The Cost in iteration 1900----->3245.2098154351